### Data Cleaning

In [20]:
import pandas as pd
file_path = r'D:\dsci\CAHOOTS\Data\call_data_from_CAD.csv'
CAD_data = pd.read_csv(file_path)

In [21]:
#CAD_data

In [22]:
#CAD_data[CAD_data["IncidentNumber"].str.contains("21000078")]

In [23]:
inconsistent_rows = CAD_data[(CAD_data['Call_First_Dispatched_Time'].notna()) & (CAD_data['Unit_Dispatched_Time'].isna())]

# Display the inconsistent rows
print(len(inconsistent_rows))

82


In [24]:
# Standardize Cahoots identifiers 
cahoots_identifiers = r"1J77\s*|3J79\s*|3J78\s*|3J77\s*|4J79\s*|3J81\s*|3J76\s*|2J28\s*|2J29\s*|CAHOOT\s*|CAHOT\s*|CAHO\s*"

CAD_data["PrimaryUnitCallSign"] = CAD_data["PrimaryUnitCallSign"].replace(cahoots_identifiers, 'CAHOOT', regex=True)
CAD_data["RespondingUnitCallSign"] = CAD_data["RespondingUnitCallSign"].replace(cahoots_identifiers, 'CAHOOT', regex=True)

# Create an identifier for Cahoots involvement 
CAD_data['Cahoots_related'] = ((CAD_data['PrimaryUnitCallSign'] == 'CAHOOT') | (CAD_data['RespondingUnitCallSign'] == 'CAHOOT')).astype(int)

# Convert dt
CAD_data["Call_Created_Time"] = pd.to_datetime(CAD_data['Call_Created_Time'], errors='coerce')
CAD_data["year"] = CAD_data["Call_Created_Time"].dt.year

# Drop unneeded cols
#data_cleaned = CAD_data.drop(columns=['Unnamed: 0', 'Beat', 'Call_Zipcode', 'Unit_Cleared_Time', 'Call_Cleared'])
CAD_data = CAD_data.drop(columns=['Unnamed: 0', 'IsPrimary'])

# Only include calls in Eugene 
zip_codes = [97402.0, 97401.0, 97405.0, 97404.0, 97403.0, 97408.0]
CAD_data = CAD_data[CAD_data['Call_Zipcode'].isin(zip_codes)]

# Drop calls with no associated unit
CAD_data = CAD_data.dropna(subset=["PrimaryUnitCallSign", "RespondingUnitCallSign"], how='all')
CAD_data = CAD_data.sort_values(by="Call_Created_Time")

# Drop Exact Duplicates
CAD_data = CAD_data.drop_duplicates()

# Drop non public calls
CAD_data = CAD_data[CAD_data["Call_Source"] != "SELF"]
CAD_data = CAD_data[CAD_data["Call_Source"] != "RPTO"]


#CAD_data

In [25]:
cahoots_data = CAD_data[CAD_data["Cahoots_related"] == 1]
#cahoots_data

In [26]:
def filter_df_by_time(df, start_time=None, end_time=None, time_column='Call_Created_Time'):
    """
    Filters the dataframe based on the time range specified by the user

    Parameters:
    df to filter
    start_time (str)
    end_time (str)
    time_column (str): The name of the column containing datetime values

    """
    df.loc[:, time_column] = pd.to_datetime(df[time_column])

    if start_time:
        if len(start_time) == 4:  # if only a year is given
            start_time = pd.to_datetime(f"{start_time}-01-01")
    else:
        start_time = df[time_column].min() 

    if end_time:
        if len(end_time) == 4:  
            end_time = pd.to_datetime(f"{end_time}-12-31 23:59:59")
    else:
        end_time = df[time_column].max()  

    filtered_df = df[(df[time_column] >= start_time) & (df[time_column] <= end_time)]

    return filtered_df

#filter_df_by_time(cahoots_data, '2021', '2021')

In [27]:
def dataset_builder(data, dispatched=False, arrived=False, cahoots_only=0, time=None):
    """
    Parameters:
    time (list of strings) [start time, end time, time_col]
    
    """
    # Apply time range if specified
    if time:
        data = filter_df_by_time(data, time[0], time[1], time[2])
    
    # Dispatched filter is based on "Unit_Dispatched_Time"
    if dispatched:
        data = data[data["Unit_Dispatched_Time"].notna()]
    
    # Arrived filter is based on "Call_First_Dispatched_Time"
    if arrived:
        data = data[data["Unit_OnScene_Time"].notna()]
        
    return data


#dataset_builder(CAD_data, dispatched=False, arrived=False, time=['2021', '2021', 'Call_Created_Time'])

In [28]:
pd.set_option('display.max_rows', 300)
testdf = dataset_builder(CAD_data, dispatched=True, arrived=True, time=['2021', '2021', 'Call_Created_Time'])

#testdf

In [29]:
#testdf[(testdf["Cahoots_related"] == 1) & (testdf["year"] == 2021)]["InitialIncidentTypeDescription"].value_counts()

In [30]:
#testdf["PrimaryUnitCallSign"].value_counts().head(200)

In [31]:
#testdf["InitialIncidentTypeDescription"].value_counts()

In [32]:
#testdf[testdf["InitialIncidentTypeDescription"] == "PURSUIT"] 

In [33]:
#testdf[testdf["InitialIncidentTypeDescription"] == "DISPUTE"].head(50)

In [34]:
#testdf[testdf["IncidentNumber"].str.contains("21001223")]

In [35]:
#testdf[testdf["InitialIncidentTypeDescription"] == "DISPUTE"]["Cahoots_related"].value_counts()

In [36]:

testdf = CAD_data[~CAD_data["IncidentNumber"].duplicated(keep=False)]
len(testdf)

262799

In [37]:
#testdf[testdf["year"]==2021]

In [38]:
#CAD_data[CAD_data["year"] == 2021]

pd.set_option('display.max_rows', None)
testdf["PrimaryUnitCallSign"].value_counts()

#testdf[testdf["Disposition"] == "FOLLOW UP INVESTIGATION"]

testdf[testdf["Disposition"] == "K893 DEPLOYED"]